In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iea-monthly-electricity-statistics/data.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import math
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch

In [3]:
df = pd.read_csv('/kaggle/input/iea-monthly-electricity-statistics/data.csv')
print(df)

              COUNTRY CODE_TIME           TIME  YEAR  MONTH MONTH_NAME  \
0           Australia   JAN2010   January 2010  2010      1    January   
1           Australia   JAN2010   January 2010  2010      1    January   
2           Australia   JAN2010   January 2010  2010      1    January   
3           Australia   JAN2010   January 2010  2010      1    January   
4           Australia   JAN2010   January 2010  2010      1    January   
...               ...       ...            ...   ...    ...        ...   
181910  United States   DEC2022  December 2022  2022     12   December   
181911  United States   DEC2022  December 2022  2022     12   December   
181912  United States   DEC2022  December 2022  2022     12   December   
181913  United States   DEC2022  December 2022  2022     12   December   
181914  United States   DEC2022  December 2022  2022     12   December   

                            PRODUCT          VALUE  DISPLAY_ORDER  \
0                             Hydro     99

In [4]:
df = df[df['PRODUCT'] == 'Solar']
print(df)

               COUNTRY CODE_TIME           TIME  YEAR  MONTH MONTH_NAME  \
2            Australia   JAN2010   January 2010  2010      1    January   
22             Austria   JAN2010   January 2010  2010      1    January   
47             Belgium   JAN2010   January 2010  2010      1    January   
72              Canada   JAN2010   January 2010  2010      1    January   
118     Czech Republic   JAN2010   January 2010  2010      1    January   
...                ...       ...            ...   ...    ...        ...   
181793           Spain   DEC2022  December 2022  2022     12   December   
181819          Sweden   DEC2022  December 2022  2022     12   December   
181843     Switzerland   DEC2022  December 2022  2022     12   December   
181866  United Kingdom   DEC2022  December 2022  2022     12   December   
181891   United States   DEC2022  December 2022  2022     12   December   

       PRODUCT        VALUE  DISPLAY_ORDER     yearToDate  previousYearToDate  \
2        Solar    

In [5]:
df = df.drop('PRODUCT', axis=1)
df = df.drop('MONTH_NAME', axis=1)
df = df.drop('TIME', axis=1)
df = df.drop('CODE_TIME', axis=1)
df = df.drop('DISPLAY_ORDER', axis=1)

In [6]:
country_encoder = LabelEncoder()
df['COUNTRY'] = country_encoder.fit_transform(df['COUNTRY'])

In [7]:
print(df)

        COUNTRY  YEAR  MONTH        VALUE     yearToDate  previousYearToDate  \
2             1  2010      1    49.216000     908.238000                 NaN   
22            2  2010      1     2.864000      84.421000                 NaN   
47            3  2010      1    17.450000     560.000000                 NaN   
72            6  2010      1     1.898000     253.000000                 NaN   
118          12  2010      1     6.205000     612.002000                 NaN   
...         ...   ...    ...          ...            ...                 ...   
181793       47  2022     12  1205.878240   32816.269966        25937.966502   
181819       48  2022     12     6.469757    1526.000000         1051.000000   
181843       49  2022     12    48.943794    3794.775534         2891.866818   
181866       50  2022     12   508.798542   14240.791059        12550.630235   
181891       51  2022     12  9172.684879  188300.466584       152294.007362   

           share  
2       0.002373  
2

In [8]:
df = df.dropna() # Removes rows with null values
df.reset_index(drop=True, inplace=True)

In [9]:
print(df)
print(df.info())

      COUNTRY  YEAR  MONTH        VALUE     yearToDate  previousYearToDate  \
0           1  2011      1   176.116000    1875.002000           49.216000   
1           2  2011      1     5.607000     165.283000            2.864000   
2           3  2011      1    19.659000    1169.000000           17.450000   
3           6  2011      1     7.373000     565.998000            1.898000   
4          12  2011      1    35.765000    2169.001000            6.205000   
...       ...   ...    ...          ...            ...                 ...   
6413       47  2022     12  1205.878240   32816.269966        25937.966502   
6414       48  2022     12     6.469757    1526.000000         1051.000000   
6415       49  2022     12    48.943794    3794.775534         2891.866818   
6416       50  2022     12   508.798542   14240.791059        12550.630235   
6417       51  2022     12  9172.684879  188300.466584       152294.007362   

         share  
0     0.007500  
1     0.000914  
2     0.0025

In [10]:
# Select columns to normalize
columns_to_normalize = ['VALUE', 'yearToDate', 'previousYearToDate', 'share']

# Create a scaler
scaler = StandardScaler()

# Normalize the selected columns
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [11]:
print(df)

      COUNTRY  YEAR  MONTH     VALUE  yearToDate  previousYearToDate     share
0           1  2011      1 -0.311911   -0.330786           -0.293959 -0.582787
1           2  2011      1 -0.339521   -0.355004           -0.295184 -0.764550
2           3  2011      1 -0.337245   -0.340787           -0.294798 -0.719497
3           6  2011      1 -0.339235   -0.349328           -0.295209 -0.786489
4          12  2011      1 -0.334637   -0.326622           -0.295095 -0.660808
...       ...   ...    ...       ...         ...                 ...       ...
6413       47  2022     12 -0.145164    0.107482            0.390069  0.677143
6414       48  2022     12 -0.339381   -0.335730           -0.267490 -0.778344
6415       49  2022     12 -0.332503   -0.303594           -0.218851 -0.534190
6416       50  2022     12 -0.258040   -0.155631            0.036351 -0.291481
6417       51  2022     12  1.144878    2.309839            3.728625 -0.104912

[6418 rows x 7 columns]


In [12]:
# Apply one-hot encoding to the 'COUNTRY' column
df = pd.get_dummies(df, columns=['COUNTRY'], prefix='country')
df = pd.get_dummies(df, columns=['MONTH'], prefix='month')
df = pd.get_dummies(df, columns=['YEAR'], prefix='year')


boolean_columns = [
    'country_0', 'country_1', 'country_2', 'country_3', 'country_4', 'country_5',
    'country_6', 'country_7', 'country_8', 'country_9', 'country_10', 'country_11',
    'country_12', 'country_13', 'country_14', 'country_15', 'country_16', 'country_17',
    'country_18', 'country_19', 'country_20', 'country_21', 'country_22', 'country_23',
    'country_24', 'country_25', 'country_26', 'country_27', 'country_28', 'country_29',
    'country_30', 'country_31', 'country_32', 'country_33', 'country_34', 'country_35',
    'country_36', 'country_37', 'country_38', 'country_39', 'country_40', 'country_41',
    'country_42', 'country_43', 'country_44', 'country_45', 'country_46', 'country_47',
    'country_48', 'country_49', 'country_50', 'country_51',
    'year_2011', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016',
    'year_2017', 'year_2018', 'year_2019', 'year_2020','year_2021','year_2022',
    'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'
   
]

df[boolean_columns] = df[boolean_columns].astype(int)

# Print the resulting dataframe
print(df)

         VALUE  yearToDate  previousYearToDate     share  country_0  \
0    -0.311911   -0.330786           -0.293959 -0.582787          0   
1    -0.339521   -0.355004           -0.295184 -0.764550          0   
2    -0.337245   -0.340787           -0.294798 -0.719497          0   
3    -0.339235   -0.349328           -0.295209 -0.786489          0   
4    -0.334637   -0.326622           -0.295095 -0.660808          0   
...        ...         ...                 ...       ...        ...   
6413 -0.145164    0.107482            0.390069  0.677143          0   
6414 -0.339381   -0.335730           -0.267490 -0.778344          0   
6415 -0.332503   -0.303594           -0.218851 -0.534190          0   
6416 -0.258040   -0.155631            0.036351 -0.291481          0   
6417  1.144878    2.309839            3.728625 -0.104912          0   

      country_1  country_2  country_3  country_4  country_5  ...  year_2013  \
0             1          0          0          0          0  ...    

In [13]:
# Model definition
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

In [14]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RegressionModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out


In [15]:
class RNNRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNRegressionModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        if len(out.shape) > 2:
            out = self.fc(out[:, -1, :])
        else:
            out = self.fc(out)
        return out.squeeze(1) if len(out.shape) > 2 else out

In [16]:
target_index = df.columns.get_loc('VALUE')

X = df.iloc[:, target_index+1:]

y = df['VALUE']

# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Number of input and output features
input_dim = X_train.shape[1]
output_dim = 1

# Initialize the model
#model = LinearRegressionModel(input_dim, output_dim)

# # Default Initialization
# nn.init.xavier_uniform_(model.linear.weight)
# nn.init.zeros_(model.linear.bias)

#model = RegressionModel(input_dim, input_dim*2, output_dim)
print(input_dim)
model = LinearRegressionModel(input_dim, output_dim)
model = RegressionModel(input_dim, input_dim*2, output_dim)
model = RNNRegressionModel(input_dim, input_dim*2, output_dim)


# Define the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)

print(df.info())

79
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6418 entries, 0 to 6417
Data columns (total 80 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   VALUE               6418 non-null   float64
 1   yearToDate          6418 non-null   float64
 2   previousYearToDate  6418 non-null   float64
 3   share               6418 non-null   float64
 4   country_0           6418 non-null   int64  
 5   country_1           6418 non-null   int64  
 6   country_2           6418 non-null   int64  
 7   country_3           6418 non-null   int64  
 8   country_4           6418 non-null   int64  
 9   country_5           6418 non-null   int64  
 10  country_6           6418 non-null   int64  
 11  country_7           6418 non-null   int64  
 12  country_8           6418 non-null   int64  
 13  country_9           6418 non-null   int64  
 14  country_10          6418 non-null   int64  
 15  country_11          6418 non-null   int64  
 16  cou

In [17]:
import wandb

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Label")

wandb.login(key=secret_value_0)

wandb.init(project='IEA Model', save_code=True)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ales-2000-09. Use `wandb login --relogin` to force relogin


In [18]:
# Training
epochs = 128
batch_size = 1
wandb.log({'Epoch': epochs})
wandb.log({'batch_size': batch_size})

for epoch in range(epochs):
    model.train()
    for i in range(0, X_train.shape[0], batch_size):
        inputs = torch.from_numpy(X_train.iloc[i:i+batch_size].values).float()
        labels = torch.from_numpy(y_train.iloc[i:i+batch_size].values).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss_total = 0.0

    with torch.no_grad():
        for i in range(0, X_val.shape[0], batch_size):
            val_batch_inputs = torch.from_numpy(X_val.iloc[i:i+batch_size].values).float()
            val_batch_labels = torch.from_numpy(y_val.iloc[i:i+batch_size].values).float().unsqueeze(1)

            val_outputs = model(val_batch_inputs)
            val_loss = criterion(val_outputs, val_batch_labels)
            val_loss_total += val_loss.item()

    avg_val_loss = val_loss_total / (X_val.shape[0] / batch_size)

    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {loss.item()}, Validation Loss: {avg_val_loss}')
    wandb.log({'Training Loss': loss.item(), 'Validation Loss': avg_val_loss})

# Test
model.eval()
with torch.no_grad():
    test_inputs = torch.from_numpy(X_test.values).float()
    test_labels = torch.from_numpy(y_test.values).float().unsqueeze(1)
    test_outputs = model(test_inputs)
    test_loss = criterion(test_outputs, test_labels)

print(f'Test Loss: {test_loss.item()}')
wandb.log({'Test Loss': test_loss.item()})

wandb.finish()

Epoch 1/128, Training Loss: 0.004873207770287991, Validation Loss: 1.2283970535166007
Epoch 2/128, Training Loss: 0.003793954150751233, Validation Loss: 1.0512887768880792
Epoch 3/128, Training Loss: 0.002449800493195653, Validation Loss: 0.9017127130750328
Epoch 4/128, Training Loss: 0.0012067186180502176, Validation Loss: 0.7741150824461103
Epoch 5/128, Training Loss: 0.0003385045565664768, Validation Loss: 0.6653483294028555
Epoch 6/128, Training Loss: 2.0176448742859066e-06, Validation Loss: 0.5733791948369941
Epoch 7/128, Training Loss: 0.0002371193841099739, Validation Loss: 0.496569376927838
Epoch 8/128, Training Loss: 0.0009875455871224403, Validation Loss: 0.4333271934959351
Epoch 9/128, Training Loss: 0.0021343855187296867, Validation Loss: 0.38199710638014384
Epoch 10/128, Training Loss: 0.0035318685695528984, Validation Loss: 0.34087910347126693
Epoch 11/128, Training Loss: 0.005036947783082724, Validation Loss: 0.30830570530499035
Epoch 12/128, Training Loss: 0.00652917893

wandb: WARNING No program path found when generating artifact job source for a non-colab notebook run. See https://docs.wandb.ai/guides/launch/create-job
wandb: WARNING Source type is set to 'artifact' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,▁
Test Loss,▁
Training Loss,▃▁▁▂▆█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████
Validation Loss,█▆▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_size,▁
Epoch,128
Test Loss,0.08876
Training Loss,0.01268
Validation Loss,0.14132
batch_size,1


In [21]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_test_true = y_test.values
y_test_pred = model(torch.from_numpy(X_test.values).float()).detach().numpy()

r2_test = r2_score(y_test_true, y_test_pred)
mse_test = mean_squared_error(y_test_true, y_test_pred)
mae_test = mean_absolute_error(y_test_true, y_test_pred)

print(f'R2 Score: {r2_test}, Mean Squared Error: {mse_test}, Mean Absolute Error: {mae_test}')

R2 Score: 0.9145332573455509, Mean Squared Error: 0.08876132358211532, Mean Absolute Error: 0.1480066987135397
